In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_ollama import ChatOllama

# Define agent state
class AgentState(TypedDict):
    user_input: str
    llm_output: str

# Initialize LLM with a better open-source model (LLaMA 3)
llm = ChatOllama(model="llama3:latest", temperature=0.0)

# Function to generate PyCaret code
def generate_pycaret_code(state: AgentState) -> AgentState:
    prompt = f"""
You are an expert Python data scientist.

Generate **complete and runnable Python code** for credit card fraud detection using PyCaret.

**Details:**
- Dataset: 'creditcard 1.csv'
- Target column: 'Class' (1 = fraud, 0 = normal)

**The code must include:**
1. Import all required libraries.
2. Load the dataset using pandas.
3. Set up PyCaret for classification.
4. Compare all classification models and select the best.
5. Train and finalize the model.
6. Predict on the dataset.
7. Print evaluation metrics and example predictions.

**Output only code, no explanations.**
    """
    try:
        response = llm.invoke(prompt)
        return {
            "user_input": state["user_input"],
            "llm_output": response.content.strip()
        }
    except Exception as e:
        return {
            "user_input": state["user_input"],
            "llm_output": f"Error generating code: {str(e)}"
        }

# Create graph
graph = StateGraph(AgentState)
graph.add_node("generate_code", generate_pycaret_code)
graph.set_entry_point("generate_code")
graph.set_finish_point("generate_code")

# Compile the app
app = graph.compile()

if __name__ == "__main__":
    user_question = "Generate fraud detection code using PyCaret"
    result = app.invoke({"user_input": user_question})

    print("\nGenerated PyCaret Code:\n")
    print(result["llm_output"])



Generated PyCaret Code:

```
import pandas as pd
from pycaret.classification import *

# Load dataset
df = pd.read_csv('creditcard 1.csv')

# Set up PyCaret for classification
setup(df, target='Class', session_id=123)

# Compare all classification models and select the best
compare()

# Train and finalize the model
finalize()

# Predict on the dataset
preds = predict()

# Print evaluation metrics and example predictions
print("Accuracy:", accuracy())
print("Precision:", precision())
print("Recall:", recall())
print("F1:", f1())
print("Example predictions:\n", preds.head(5))
```
